# **Machine Learning Group Project: Team 7**
# Project introduction

| Student Name | Student Number | Class Group |
| --- | --- | --- |
| **Alessandro Maugeri** | 53067 | TA |
| **Frank Andreas Bauer** | 53121 | TA |
|  **Johannes Rahn** | 53958 | TB |
| **Nicole Zoppi** | 53854 | TB |
| **Yannick von der Heyden** | 53629 | TA |



- **Course & Course Code:** Machine Learning (2487)
- **Instructors:** Qiwei Han & Alessandro Gambetti
- **Academic Year:** 2022/2023


This project was developed by Alessandro Alfio Maugeri, Frank Andreas Bauer, Johannes Rahn, Nicole Zoppi, and Yannick von der Heyden as part of Nova SBE's Machine Learning course taught during Semester 2 of the 2022/2023 academic year by Qiwei Han and Alessandro Gambetti. The purpose of the group work is to develop an end-to-end Machine Learning project with data of our choice, clearly highlighting the business use for our model.

The project exploits data from two distinct Kaggle pages ("Game Recommendations on Steam" and "Steam Store Games (Clean dataset)") which contain data on content published to the **Steam libary** (e.g. games and DLCs) and user-generated content associated with it (e.g. whether a review is positive or not, what general consensus is, how many people found the review useful, etc.). The final work merges data from 4 distinct CSV files and 1 JSON, all of which can be found in the data folder and all of whose contents are covered in detail in the current Notebook.

The objective is to develop a Machine Learning pipeline from beginning to end whose output has a clear, pertinent, and useful business use. In this project, we explore multiple models in order to identify the one which is best-suited for **predicting whether a given Steam user will enjoy a game or not**. This should allow Steam to optimize its service by tailoring it to individual clients and fine-tuning their recommender systems.

To guarantee a better computing perfomance, we split the project steps into 5 distinct Notebooks, meant to be run in succession. 

1. **a_csv_creation**. The current notebook presents the outline of the projects and reads data from the various datasets used, merging them into a final one (final_df) used for the following Notebooks.
2. **b_recommender_system**. The purpose of this notebook is to extract content-based similarities across different products on the basis of the plethora of of textual data contained in descriptive columns such as tags, genres, description, or steamspy_tags. These findings can later be utilized when building other models.
3. **c_network_analytics** TO ADD
4. **d_data_exploration**. The goal of the notebook is to display an in-depth data exploration, critical to make informed decisions about how to preprocess the data, select appropriate features, and choose the best machine learning algorithm for the problem.
5. **e_models**. The notebook presents the different model tested on the data, the best performing selected one, and the predicitions obtain by it.


----

# CSV File creation
**Please Note:** This notebook's purpose is to read data from the various datatsets used in our analysis. The data subsequently undergoes some changes and is merged into a single DataFrame `final_df`. The final snippet of code saves this new DataFrame as a CSV file in the _data_ folder. [Notebook _b_data_exploration_](b_data_exploration.ipynb) then works with this data by importing this CSV. Thus, it is not necessary to run this Notebook every time the code is run on a given machine. One time is sufficient to view the process of importing and merging and store the data in a local directory for later use.

## Importing Packages

In [1]:
import ast
import pandas as pd
import numpy as np
from datetime import datetime

## Importing Data

The data for this project was retrieved from [Kaggle](https://www.kaggle.com/datasets/antonkozyriev/game-recommendations-on-steam?select=games.csv) and stored in the "data" folder found in the notebook's directory. The folder includes **five data files**:

The CSV file **[games.csv](data/games.csv)** presents data concerning individual games in the Steam library:

| Column | Description | Example|
| --- | --- | --- |
| **app_id** | Product ID on Steam | 620 |
| **title** | Product Commercial Title | Portal 2|
|  **date_release** | Release Date of Title (y-m-d) | 2011-04-18 |
| **win** | Boolean Denoting Compatibility to Windows Computers | True |
| **mac** | Boolean Denoting Compatibility to Mac Computers  | True | 
| **linux** | Boolean Denoting Compatibility to Linux Computers  | True |
| **rating** | Categorical Rating of Product (e.g. "Positive")| Overwhelmingly Positive |
| **positive_ratio** | Ratio of Postive Feedback for Game  | 98 |
| **user_reviews** | Number of Reviews  | 267142 |
| **price_final** | Final Price in USD | 9.99 |
| **price_original** | Price Before Discounts in USD | 9.99 |
| **discount** | Discount Percentage Applied | 0 |
| **steam_deck** | Compatible with [Steam Deck](https://store.steampowered.com/steamdeck)? | True |



In [2]:
df_games_data = pd.read_csv("data/games.csv")
df_games_data.head(2)

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,10090,Call of Duty: World at War,2008-11-18,True,False,False,Very Positive,92,37039,19.99,19.99,0.0,True
1,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True


----
The **CSV file [users.csv](data/users.csv)** presents data concerning individual users found in the datasets:

| Column | Description | Example|
| --- | --- | --- |
| **user_id** | User ID on Steam | 5693478 |
| **products** | Number of Products from Steam Library Owned | 156 |
|  **reviews** | Number of Reviews Published | 1 |

In [3]:
df_users = pd.read_csv("data/users.csv")
df_users.head(2)

,user_id,products,reviews
0,5693478,156,1
1,3595958,329,3


----
The **CSV file [recommendations.csv](data/recommendations.csv)** has a many-to-many relationship to both users.csv and games.csv and contains data concerning user reviews of specific games:

| Column | Description | Example|
| --- | --- | --- |
| **app_id** | Product ID on Steam | 620 |
| **helpful** | Number of Users Who Found Review Helpful | 0 |
|  **funny** | Number of Users Who Found Review Funny | 0 |
| **date** | Date in Which Review was Published (y-m-d) | 2022-12-12 |
| **is_recommended** | Does the User Recommend the Title | True | 
| **hours** | Hours Spent by User Playing Game  | 36.3 |
| **user_id** | User ID of Review Author | 19954 |
| **review_id** | ID of Individual Review  | 0 |

Please note that this file is too large to upload via Moodle (even when zipped) or on a Git repository. Consequently, in order to run the cell below, the recommendations.csv file must be [downloaded from Kaggle](https://www.kaggle.com/datasets/antonkozyriev/game-recommendations-on-steam?select=games.csv) and added to the data folder.

In [4]:
df_recommendations = pd.read_csv("data/recommendations.csv")
df_recommendations.head(2)

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id
0,975370,0,0,2022-12-12,True,36.3,19954,0
1,304390,4,0,2017-02-17,False,11.5,1098,1


----
Finally, the folder includes a **JSON file [games_metadata.json](data/games_metadata.json)** containing metadata on individual games:

| Column | Description | Example|
| --- | --- | --- |
| **app_id** | Product ID on Steam | 304430 |
| **description** | Game Description on Steam | "Hunted and alone, a boy finds himself drawn into the center of a dark project. INSIDE is a dark, narrative-driven platformer combining intense action with challenging puzzles. It has been critically acclaimed for its moody art style, ambient soundtrack and unsettling atmosphere." |
|  **tags** | Additional Tags on Steam Platform | ["2.5D", "Story Rich", "Puzzle Platformer" , "Atmospheric" , "Adventure" , "Indie" , "Dark" , "Horror" , "Singleplayer" , "Action-Adventure" , "Puzzle" , "Multiple Endings" , "Exploration" , "2D Platformer" , "Platformer" , "Controller" , "Soundtrack" , "Ambient" , "Action" , "Narrative"] |

In [5]:
df_games_meta_data = pd.read_json('data/games_metadata.json', lines=True)
df_games_meta_data.head(2)

,app_id,description,tags
0,10090,"Call of Duty is back, redefining war like you'...","[Zombies, World War II, FPS, Multiplayer, Acti..."
1,13500,Enter the dark underworld of Prince of Persia ...,"[Action, Adventure, Parkour, Third Person, Gre..."


----
Two additional data files are utilized from a separate [Kaggle page](https://www.kaggle.com/datasets/nikdavis/steam-store-games?select=steam.csv) to enrich the analysis. These are imported below.


The **CSV file [steam.csv](data/steam.csv)** provides additional data on the games:

| Column | Description | Example|
| --- | --- | --- |
| **appid** | Product ID on Steam| 10 |
| **name** | Name of Game | Counter-Strike |
|  **release_date** | Release Date of Title (y-m-d) | 2000-11-01 |
|  **english** | Is the Game Available in English? | 1 |
|  **developer** | Developer Company of Game | Valve |
|  **publisher** | Publishing Company of Game | Valve |
|  **platforms** | Semicolon-Delimited List of Systems that Can Run the Game | windows;mac;linux |
|  **required_age** | Minimum Age Required to Play by PEGI UK (0 Means Unsupplied) | 0 |
|  **categories** | Game Categorisation (Semicolon Delimited) | Multi-player;Online Multi-Player... |
|  **genres** | Game's Genre (Semicolon Delimteted) | Action |
|  **steamspy_tags** | Tags from [Steamspy API](https://steamspy.com/) (Semicolon Delimited) | Action;FPS;Multiplayer |
|  **achievements** | Number of In-Game Achievements (If Any) | 0 |
|  **positive_ratings** | Number of Positive Ratings | 124534 |
|  **negative_ratings** | Number of Negative Ratings | 3339 |
|  **average_playtime** | Average Playtime by User in Minutes| 17612 |
|  **median_playtime** | Median Playtime by User in Minutes | 317 |
|  **owners** | Number of Users that Own the Game (Bracket) | 10000000-20000000 |
|  **price** | Price of Game | 7.19 |



In [6]:
df_games_additional = pd.read_csv("data/steam.csv")
df_games_additional.head(2)

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99


## Preliminary Data Exploration

**Rating Values:** We find that the _rating_ columns can take on any of the values below. It is worth noting that these ratings are [not solely based on the number of positive and negative reviews](https://www.gamedeveloper.com/business/fixing-steam-s-user-rating-charts). The rating system seems to "boost" the ratings of all those titles which have many reviews. For instance, a title with extremely positive feedback but a small number of reviews can only ever classify as "Positive" (and vice versa, one with extremely negative reviews but a small number of reviews can only ever classify as "Negative"). In order to qualify as "Very Positive", "Very Negative", "Overwhelmingly Positive", or "Overwhelmingly Negative", a title must reach certain review thresholds. While the precise thresholds are unknonwn, it is known that the ratings can be ordered in descending positivity as follows: Overwhelmingly Positive, Very Positive, Positive, Mostly Positive, Mixed, Mostly Negative, Negative; Very Negative, Overwhelmingly Negative.

In [7]:
df_games_data["rating"].unique()

array(['Very Positive', 'Positive', 'Mixed', 'Mostly Positive',
       'Overwhelmingly Positive', 'Mostly Negative',
       'Overwhelmingly Negative', 'Negative', 'Very Negative'],
      dtype=object)

**Owner Count:** The _owners_ column can take on 13 different values and is expressed in brackets rather than exact numbers. Hence, the datatype is string. These come in the following order: 0-20000, 20000-50000, 50000-100000, 100000-200000, 200000-500000, 500000-1000000, 1000000-2000000, 2000000-5000000, 5000000-10000000, 10000000-20000000, 20000000-50000000, 50000000-100000000, 100000000-200000000

In [8]:
df_games_additional["owners"].unique()

array(['10000000-20000000', '5000000-10000000', '2000000-5000000',
       '20000000-50000000', '100000000-200000000', '50000000-100000000',
       '20000-50000', '500000-1000000', '100000-200000', '50000-100000',
       '1000000-2000000', '200000-500000', '0-20000'], dtype=object)

**Developer Count:** The _developer_ column features 17,113 different developers. We see that a few of these developers appear few times, we display how many do so for any number of appearances between 1 and 5.

In [9]:
df_games_additional["developer"].value_counts()

Choice of Games               94
KOEI TECMO GAMES CO., LTD.    72
Ripknot Systems               62
Laush Dmitriy Sergeevich      51
Nikita "Ghost_RUS"            50
                              ..
CRAPPY ZOMBIE GAME STUDIO      1
Ramon Mujica                   1
Oomst Games                    1
Joe Censored Games             1
Adept Studios GD               1
Name: developer, Length: 17113, dtype: int64

In [10]:
print("The number of developers which appear only once is:", 
      len(df_games_additional["developer"].value_counts()[df_games_additional["developer"].value_counts()==1]))

for i in range(2, 6):
    print(f"The number of developers which appear {i} times is:", 
      len(df_games_additional["developer"].value_counts()[df_games_additional["developer"].value_counts() == i]))

The number of developers which appear only once is: 13266
The number of developers which appear 2 times is: 2105
The number of developers which appear 3 times is: 765
The number of developers which appear 4 times is: 341
The number of developers which appear 5 times is: 179


## Data Preparation

#### Games Data
- Turning relevant columns to datetime objects

In [11]:
# Turn date_release column to Pandas DateTime
df_games_data["date_release"] = pd.to_datetime(df_games_data["date_release"])

#### Additional Game Data
- Drop features that are redundant with df_games_data
- Convert number of owners to being counted in 1000s

In [12]:
# Drop Redundant Columns
df_games_additional.drop(["name","platforms", "release_date", "price", 
                          "positive_ratings", "negative_ratings"], axis = 1, inplace = True)

# 1000s Conversion
df_games_additional["owners"].replace({"0-20000": "0-20",
                                       "20000-50000": "20-50",
                                       "50000-100000": "50-100",
                                       "100000-200000": "100-200",
                                       "200000-500000": "200-500",
                                       "500000-1000000": "500-1000",
                                       "1000000-2000000": "1000-2000",
                                       "2000000-5000000": "2000-5000",
                                       "5000000-10000000": "5000-10000",
                                       "10000000-20000000": "10000-20000",
                                       "20000000-50000000": "20000-50000",
                                       "50000000-100000000": "50000-100000",
                                       "100000000-200000000": "100000-200000"
                                      }, inplace=True)

In [13]:
df_games_additional.head(1)

,appid,english,developer,publisher,required_age,categories,genres,steamspy_tags,achievements,average_playtime,median_playtime,owners
0,10,1,Valve,Valve,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,17612,317,10000-20000


#### Recommendations Data
- Turn date features into datetime objects
- Rename is_recommended to y, to signal it is the target variable

In [14]:
# Turn to DateTime
df_recommendations["date"] = pd.to_datetime(df_recommendations["date"])

# Rename column
df_recommendations.rename(columns = {"is_recommended": "y"}, inplace = True)

#### Games Metadata

In [15]:
# Turn the Tags Column Into a List
df_games_meta_data["tags"] = df_games_meta_data["tags"].astype(str).apply(ast.literal_eval)

## Merging Datasets

We merge the data into one single final DataFrame.

In [16]:
# Merge all information on games to one DataFrame
games_df = df_games_data.merge(df_games_meta_data, how = "inner")
games_df = games_df.merge(df_games_additional, how = "inner",
                          left_on = "app_id", right_on = "appid")

# Merge game information into the recommendations DataFrame
recs_df = df_recommendations.merge(games_df, how = "inner", on = "app_id")

# Merge all information on users into a final DataFrame
final_df = recs_df.merge(df_users, how="inner", on = "user_id")

## Final dataset overview

In [18]:
# Rearrange Columns
final_df = final_df.reindex(columns = ["review_id", "y", "date", "funny", "helpful",
                                       "app_id", "title", "date_release", "rating", 
                                       "positive_ratio", "user_reviews", "win", "mac", "linux", 
                                       "price_final", "price_original", "discount", "description",
                                       "tags", 'english', 'developer', 'publisher', 'required_age', 
                                       'genres', 'steamspy_tags','achievements', 'average_playtime', 
                                       'median_playtime', "owners", "steam_deck",
                                       "user_id", "products", "reviews", "hours", "categories"])

# Sort by Review ID
final_df.sort_values(by=["review_id"], inplace = True)

In [19]:
# Display the final dataset
final_df.head(3)

,review_id,y,date,funny,helpful,app_id,title,date_release,rating,positive_ratio,...,achievements,average_playtime,median_playtime,owners,steam_deck,user_id,products,reviews,hours,categories
0,1,False,2017-02-17,0,4,304390,FOR HONOR™,2017-02-13,Mixed,68,...,0,951,696,2000-5000,True,1098,269,1,11.5,Single-player;Online Multi-Player;Online Co-op...
174074,5,True,2021-10-10,0,0,306130,The Elder Scrolls® Online,2017-05-22,Very Positive,83,...,0,10659,3143,1000-2000,True,17622,15,4,8.6,MMO;Steam Trading Cards;Partial Controller Sup...
355245,6,True,2017-11-25,0,0,238960,Path of Exile,2013-10-23,Very Positive,87,...,120,5263,492,10000-20000,True,33969,273,1,538.8,Single-player;Multi-player;Online Multi-Player...


In [20]:
# Number of Observations and Features
print(f"The final dataset has \033[1m{final_df.shape[0]} observations\033[0m and \033[1m{final_df.shape[1]} features.\033[0m")

# Number of Missing Valyes
print(f"The final dataset has \033[1m{final_df.isna().sum().sum()} NaN entries.\033[0m")

The final dataset has 6564923 observations and 35 features.
The final dataset has 0 NaN entries.


In [21]:
# General DataFrame Description
final_df.describe(percentiles=[.1, .25, .5, .75, .9]).round(2).T

,count,mean,std,min,10%,25%,50%,75%,90%,max
review_id,6564923.0,5136968.74,2967648.97,1.0,1024397.2,2559700.00,5120629.00,7756637.50,9286506.60,10072269.00
funny,6564923.0,1.39,48.06,0.0,0.0,0.00,0.00,0.00,0.00,28088.00
helpful,6564923.0,3.46,58.34,0.0,0.0,0.00,0.00,0.00,3.00,29627.00
app_id,6564923.0,333103.39,185197.05,70.0,49520.0,238960.00,294100.00,431960.00,594650.00,1032430.00
positive_ratio,6564923.0,87.83,7.84,44.0,78.0,83.00,88.00,94.00,97.00,98.00
user_reviews,6564923.0,415855.42,1132509.05,174.0,21086.0,52226.00,105614.00,204840.00,722973.00,6941137.00
price_final,6564923.0,21.18,15.57,0.0,0.0,9.99,19.99,29.99,39.99,59.99
price_original,6564923.0,22.42,16.51,0.0,0.0,9.99,19.99,29.99,39.99,109.99
discount,6564923.0,4.32,17.51,0.0,0.0,0.00,0.00,0.00,0.00,90.00
english,6564923.0,1.00,0.00,1.0,1.0,1.00,1.00,1.00,1.00,1.00


In [22]:
# Variable types
final_df.dtypes

review_id                    int64
y                             bool
date                datetime64[ns]
funny                        int64
helpful                      int64
app_id                       int64
title                       object
date_release        datetime64[ns]
rating                      object
positive_ratio               int64
user_reviews                 int64
win                           bool
mac                           bool
linux                         bool
price_final                float64
price_original             float64
discount                   float64
description                 object
tags                        object
english                      int64
developer                   object
publisher                   object
required_age                 int64
genres                      object
steamspy_tags               object
achievements                 int64
average_playtime             int64
median_playtime              int64
owners              

## Dataset storing
After creating and briefly analyzing the final dataframe, we create _reduced_df_: a reduced version of it with 500000 records. The goal is to improve the dataset usability and decrease the computation time. The user can decide which version to download and store in the _data_ folder.

In [23]:
# Subset the dataframe
reduced_final_df = final_df.sample(n=500000, random_state=1)

In [24]:
# Save reduced_final_df to file
reduced_final_df.to_csv('data/reduced_final_df.csv', index = False)

In [ ]:
# Save final_df to file
final_df.to_csv('data/final_df.csv', index = False)